<a href="https://colab.research.google.com/github/Tubhalooter/codeschool/blob/tss/TSS_JulyProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the 30 days of code challenge!
___
Feel free to add your code files you have so far in the cells below.
If you need to install any packages for python, you can do below in a code cell by running:
```
!pip install package-name --upgrade --quiet
```
--upgrade is added so it installs the latest package versions and --quiet so it doesn't spam all over the output cell, it will only show error messages

In [2]:
!pip install selenium

     |████████████████████████████████| 904 kB 6.5 MB/s 


In [3]:

#ls code
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

# libraries imported^

# what page to use
url = 'https://spaceflightnow.com/launch-schedule/'

# request html text from page
html_text = requests.get(url).text

# lxml as parser(finds and then structures/makes sense of html text) to extract html code
html_S_code = BeautifulSoup(html_text, 'lxml')

# this uses the line above and we put in these arguments to then find the line of html code from the page

missionDate = html_S_code.find_all('div', class_="datename")
# to find mission date   # ^ find_all will find all instances of that type inside the brackets


missionData = html_S_code.find_all('div', class_="missiondata")
# to find info of mission

missionDesc = html_S_code.find_all('div', class_="missdescrip")
# to find description of mission

# the three variables above now hold all the lines of html code that match with their arguments


map = {}
id = -1  # for dictionary index counter inside for loop
num_mission = len(missionDate)

for i in range(num_mission):
    dateC = missionDate[i]
    dataC = missionData[i]
    descC = missionDesc[i]

    id += 1
    map[id] = {}

    lDate = dateC.text
    lData = dataC.text
    lDesc = descC.text

    # mapping to a location(id) in the dictionary below
    map[id]['Mission Date'] = lDate
    map[id]['Mission Data'] = lData
    map[id]['Mission Description'] = lDesc
    map[id]['mission num'] = []  # here we have created an empty list that is inside the dictionary under the category mission num, list in a dictionary
    # all of these^ are in the same location in the dictionary, just under different categories in same location

    # all of our scraped values are in a dictionary right now and need to be converted to a df
    # we need to convert to a list of lists to do this

    list = []  # list created

for id in range(len(missionDate)):  # the reason this works is because the parameter inside the len() points to a list,
    # len() of a list gives amount of values
    list.append([])  # appending the list

    list[id].append(str(map[id][
                            'Mission Date']))  # appending a certain value of the list with the string version of the value in (map[id]['missionData'])
    list[id].append(str(map[id]['Mission Data']))
    list[id].append(str(map[id]['Mission Description']))

events_to_df = pd.DataFrame(list, columns=['Mission Date', 'Mission Info', 'Mission Desc'])

MissionFile = 'MissionFiles.csv'
events_to_df.to_csv(MissionFile)

events_df = pd.read_csv('MissionFiles.csv')
events_df

cleaned_data = events_df.replace('\n', ' ', regex=True)# to replace data in a data frame, pretty self explanatory

cleaned_data

,Unnamed: 0,Mission Date,Mission Info,Mission Desc
0,0,July 21Proton • Nauka,Launch time: 1458 GMT (10:58 a.m. EDT) Launch ...,A Russian government Proton rocket will launch...
1,1,JulyFalcon 9 • Starlink,"Launch time: TBD Launch site: SLC-4E, Vandenbe...",A SpaceX Falcon 9 rocket will launch on the fi...
2,2,July 27Ariane 5 • Star One D2 & Eutelsat Quantum,"Launch window: TBD Launch site: ELA-3, Kourou,...","Arianespace will use an Ariane 5 ECA rocket, d..."
3,3,July 30Atlas 5 • CST-100 Starliner Orbital Fli...,Launch time: 1853 GMT (2:53 p.m. EDT) Launch s...,"A United Launch Alliance Atlas 5 rocket, desig..."
4,4,Aug. 2Soyuz • OneWeb 9,Launch time: TBD Launch site: Baikonur Cosmodr...,A Russian Soyuz rocket will launch 34 satellit...
5,5,Aug. 10Antares • NG-16,Launch time: 2155 GMT (5:55 p.m. EDT) Launch s...,A Northrop Grumman Antares rocket will launch ...
6,6,Mid-AugustVega • Pléiades Neo 4,"Launch time: TBD Launch site: ZLV, Kourou, Fre...","An Arianespace Vega rocket, designated VV19, w..."
7,7,Aug. 18Falcon 9 • SpaceX CRS 23,"Launch time: TBD Launch site: LC-39A, Kennedy ...",A SpaceX Falcon 9 rocket will launch a Dragon ...
8,8,Aug. 26Soyuz • OneWeb 10,Launch time: TBD Launch site: Baikonur Cosmodr...,A Russian Soyuz rocket will launch 34 satellit...
9,9,Early SeptemberAtlas 5 • STP-3,"Launch time: TBD Launch site: SLC-41, Cape Can...",A United Launch Alliance Atlas 5 rocket will l...


In [5]:
import urllib.parse
from bs4 import BeautifulSoup


launchSiteList=[]
for listloc in range(len(missionData)):
    launchSiteList.append([])
    find_launch_sites = str(events_df['Mission Info'][listloc]).split('\n')[1]
    launchSiteList[listloc].append(find_launch_sites)


LaunchSite_dataframe = pd.DataFrame(launchSiteList, columns = ['Launch Sites'])
LaunchS_File = 'Launch Sites List.csv'
LaunchSite_dataframe.to_csv(LaunchS_File)

LaunchS_File_R = pd.read_csv('Launch Sites List.csv')

In [6]:
import geopy
from geopy import ArcGIS
from geopy.extra.rate_limiter import RateLimiter
LaunchS_File_R

LSlistDF=pd.read_csv('Launch Sites List.csv')

locator = ArcGIS()
geocode = RateLimiter(locator.geocode,min_delay_seconds=1)
LSlistDF['Co-ordF'] = LSlistDF['Launch Sites'].apply(geocode)
LSlistDF['point'] = LSlistDF['Co-ordF'].apply(lambda loc: tuple(loc.point) if loc else None)

co_ordDF=pd.DataFrame(LSlistDF['point'].tolist(), index=LSlistDF.index,columns=["latitude","longitude",'altitude'])
co_ordDF

,latitude,longitude,altitude
0,45.616670,63.316670,0.0
1,38.271649,-121.927679,0.0
2,5.168300,-52.645560,0.0
3,28.477630,-80.587910,0.0
4,45.616670,63.316670,0.0
5,37.844040,-75.479110,0.0
6,5.168300,-52.645560,0.0
7,28.526140,-80.681360,0.0
8,45.616670,63.316670,0.0
9,28.394549,-80.606196,0.0


In [28]:
import folium
LSmaps = folium.Map(
    location=[40,0],
    tiles='cartoDB dark_matter',
    zoom_start=2.4,
)
co_ordDF.apply(lambda row:folium.vector_layers.CircleMarker(location=tuple([row['latitude'],row['longitude']])).add_to(LSmaps),axis=1)
LSmaps



